
# Day 14 AoC

🕎 [Day 14 description](https://adventofcode.com/2022/day/14) 🕎


## Setup

In [ ]:
# imports
import os, re, sys, IPython, itertools, operator, functools, datetime, heapq

starttime = datetime.datetime.now()

In [ ]:
# common helper, data import
def ans(val):
    return IPython.display.Markdown("**Answer: {}**".format(val))

data_fd = open('inputs/input-aoc-22-14.txt', 'r')
data = data_fd.read().strip().split('\n')

In [ ]:
data[0]

In [ ]:
class AOCMap(object):
    def __init__(self, data, use_floor=False):
        self._data = data
        self._rock_paths = []
        self._rock_coords = set()
        self._sand_coords = set()
        self._max_coord = None
        self._min_coord = None
        self._sand_start = (500,0)
        self._sand_path = []
        self._use_floor = use_floor
        self._parse_data()
        
    def _parse_data(self):
        for line in self._data:
            path = []
            for coordstr in line.split(' -> '):
                x,y = coordstr.split(',')
                path.append((int(x), int(y)))
            self._rock_paths.append(path)
            
        for path in self._rock_paths:
            for i in range(len(path)-1):
                pstart = path[i]
                pend = path[i+1]

                if pstart[0] == pend[0]:
                    direction = (pend[1] - pstart[1])//abs(pend[1] - pstart[1])
                    for j in range(pstart[1],pend[1]+direction, direction):
                        self._rock_coords.add((pstart[0], j))
                elif pstart[1] == pend[1]:
                    direction = (pend[0] - pstart[0])//abs(pend[0]-pstart[0])
                    for j in range(pstart[0],pend[0]+direction, direction):
                        self._rock_coords.add((j, pstart[1]))
                else:
                    print("bad assumption")
        floor_adj = 0
        if self._use_floor:
            floor_adj = 2
        self._max_coord = (max([x[0] for x in self._rock_coords]),
                           max([x[1]+floor_adj for x in self._rock_coords]))
        self._min_coord = (min([x[0] for x in self._rock_coords]),
                           min([x[1] for x in self._rock_coords]))

    def _free_coord(self, coord):
        result = ( coord not in self._rock_coords and \
                   coord not in self._sand_coords )
        if not self._use_floor:
            return result
        else:
            return ( coord[1] < self._max_coord[1] and
                     result )
    
    def _oob(self, coord):
        result = \
               coord[1] < 0 or \
               coord[1] > self._max_coord[1]
        if not self._use_floor:
            return result
        else:
            return result or (coord == (500,0))     

    def _find_next_option(self, sand_coord):
        options = [(sand_coord[0]+1, sand_coord[1]+1),
                   (sand_coord[0]-1, sand_coord[1]+1),
                   (sand_coord[0], sand_coord[1]+1)]
        
        while len(options) > 0:
            opt = options.pop()
            if self._free_coord(opt):
                return opt
        return None
    
    def sim_sand(self):
        keep_going = True
        sand_coord = self._sand_start
        if len(self._sand_path) > 0:
            sand_coord = self._sand_path[-1]
        try:
            while keep_going:
                opt = self._find_next_option(sand_coord)
                if opt == None:
                    keep_going = False
                    self._sand_coords.add(sand_coord)
                    if len(self._sand_path) > 0:
                        self._sand_path.pop()
                elif self._oob(opt):
                    keep_going = False
                    print("one fell off! at {}".format(len(self._sand_coords)))
                else:
                    sand_coord = opt
                    self._sand_path.append(sand_coord)
        except KeyboardInterrupt:
            print("sand coord: {}".format(repr(sand_coord)))
            raise
    def run_sim(self, max_count=None):
        sand_count = len(self._sand_coords)
        self.sim_sand()
        next_count = len(self._sand_coords)
        while next_count != sand_count and (max_count==None or sand_count<=max_count):
            sand_count = len(self._sand_coords)
            self.sim_sand()
            next_count = len(self._sand_coords)
        return len(self._sand_coords)
        
    def print_map(self):
        result = []
        for y in range(0, self._max_coord[1]+1):
            resline = ''
            for x in range(self._min_coord[0], self._max_coord[0]+1):
                if self._use_floor and y == self._max_coord[1]:
                    resline = resline+"#"
                elif (x,y) in self._rock_coords:
                    resline = resline+"#"
                elif (x,y) in self._sand_coords:
                    resline = resline+"o"
                elif (x,y) == self._sand_start:
                    resline = resline+"+"
                else:
                    resline = resline+'.'
            result.append(resline)
        print('\n'.join(result))

## Part 1

In [ ]:
m = AOCMap(data)
v = m.run_sim(750)
m.print_map()
ans(v)

## Part 2

In [ ]:
m = AOCMap(data, use_floor=True)
v = m.run_sim(50000)
m.print_map()
ans(v)

In [ ]:
endtime = datetime.datetime.now()

print(endtime - starttime)

## Notes


Good problem for optimization. Basic path cache took the runtime from 5.9 sec to .44 sec on my laptop.

## Bugs



1. Bad rock calculation. Using range produced `[]` when going negative direction (fixed by adding the step parameter to range). Similarly, adding 1 to the end of the range to be inclusive of the endpoint needed the direction as well.
2. Out of bounds check. Used `and` instead of `or`, dumb logic error

In [ ]:
test = AOCMap("""498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9""".split('\n'))
v = test.run_sim()
test.print_map()
print((497,6) in test._rock_coords)
print(test._min_coord)
print(test._max_coord)
print(test._rock_paths)